In [13]:
# %matplotlib widget
import csv
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os 
import json
from decimal import Decimal, ROUND_HALF_UP
import math

class Test:
    def __init__(self, label_path):
        self.label_path = label_path
        self.label = []
        self.grid_points = []
        self.bboxes = []
        self.x1 = self.y1 = self.x2 = self.y2 = self.x3 = self.y3 = self.x4 = self.y4 = 0
        self.rows = 0
        self.cols = 0
    def interpolate(self, p1, p2, steps):
        return [
            [p1[0] + (p2[0] - p1[0]) * i / steps,
            p1[1] + (p2[1] - p1[1]) * i / steps]
            for i in range(steps + 1)
        ]
    def calculate_grid(self, x1, y1, x2, y2, x3, y3, x4, y4):
        vertical_divisions = self.rows
        
        horizontal_divisions = self.cols
        corners = [
                    [x1, y1],       
                    [x2, y2],       
                    [x3, y3],       
                    [x4, y4]        
                ]
        tl, bl, br, tr = corners
        
        left_edge = self.interpolate(tl, bl, vertical_divisions)
        right_edge = self.interpolate(tr, br, vertical_divisions)
        print(left_edge, right_edge)
        grid = []
        for i in range(vertical_divisions + 1):
            row = self.interpolate(left_edge[i], right_edge[i], horizontal_divisions)
            grid.append(row)
        
        return grid

    def get_bboxes(self, rows, cols):
        with open(self.label_path, 'r') as file:
            data = json.load(file)
            for shape in data.get('shapes', []):
                if shape.get('label') == 'master_box':
                    bbox = shape.get('points')
                    print(bbox)
                    for i in bbox:
                        for j in i:
                            self.label.append(j)
        self.rows, self.cols = rows, cols

        self.x1, self.y1, self.x2, self.y2, self.x3, self.y3, self.x4, self.y4 = self.label[:8]
        grid = self.calculate_grid(self.x1, self.y1, self.x2, self.y2, self.x3, self.y3, self.x4, self.y4)
        return grid

    def save_squares_to_csv(self, grid, filename):
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Square_ID', 'Top_Left_X', 'Top_Left_Y','Bottom_Left_X', 'Bottom_Left_Y', 'Top_Right_X', 'Top_Right_Y', 
                             'Bottom_Right_X', 'Bottom_Right_Y'])
            
            square_id = 1
            for row in range(len(grid) - 1):
                for col in range(len(grid[0]) - 1):
                    top_left = grid[row][col]
                    top_right = grid[row][col+1]
                    bottom_left = grid[row+1][col]
                    bottom_right = grid[row+1][col+1]
                    
                    writer.writerow([
                        square_id,
                        top_left[0], top_left[1],
                        bottom_left[0], bottom_left[1],
                        bottom_right[0], bottom_right[1],
                        top_right[0], top_right[1]
                    ])
                    
                    square_id += 1



In [14]:

def calculate_dimension( x1, y1, x2, y2, x3, y3, x4, y4):
        left = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        right = math.sqrt((x4 - x3) ** 2 + (y4 - y3) ** 2)
        butom  = math.sqrt((x3 - x2) ** 2 + (y3 - y2) ** 2)
        top = math.sqrt((x4 - x1) ** 2+ (y4 - y1) ** 2)
        return left, right, butom, top        
        
def calculate_number_of_boxes( total_boxes, bboxes):
    num_of_box_top = 0
    num_of_box_left = 0
    # top_left, bottom_left, bottom_right, top_right
    _x1, _y1, _x2, _y2, _x3, _y3, _x4, _y4 = bboxes
    x1, y1, x2, y2, x3, y3, x4, y4 = total_boxes

    total_left, total_right, total_butom, total_top = calculate_dimension(x1, y1, x2, y2, x3, y3, x4, y4)

    left, right, butom, top = calculate_dimension(_x1, _y1, _x2, _y2, _x3, _y3, _x4, _y4)

    
    num_slice_left = total_left / left
    num_slice_right = total_right / right
    num_slice_butom = total_butom / butom
    num_slice_top = total_top / top
    def custom_round(num):
        if num - int(num) > 0.5:
            return int(num) + 1
        else:
            return int(num)
    
    num_of_box_left = (num_slice_left+num_slice_right)/2
    num_of_box_top = (num_slice_butom+num_slice_top)/2
    
    num_of_box_left = custom_round(num_of_box_left)
    num_of_box_top = custom_round(num_of_box_top)

    
    return num_of_box_left, num_of_box_top


In [15]:

def get_bboxes(label_path):
    chilrenbbox = []
    masterbbox = []
    with open(label_path, 'r') as file:
        data = json.load(file)
        for shape in data.get('shapes', []):
            label = shape.get('label')
            if label == "black_box":
                bbox = shape.get('points')
                for i in bbox:
                    for j in i:
                        chilrenbbox.append(j)
            if label == "master_box":
                bbox = shape.get('points')
                for i in bbox:
                    for j in i:
                        masterbbox.append(j)
            if label == "numb_box":
                bbox = shape.get('points')
                for i in bbox:
                    for j in i:
                        chilrenbbox.append(j)
            
                        
    return masterbbox, chilrenbbox



In [16]:

def save_number_of_boxes(rows, cols, an_path):
    data = [
        rows, cols, an_path
    ]
    return data

In [17]:
import re
from collections import defaultdict

def avg_colume(data):
    folder_sums = defaultdict(lambda: [0, 0, 0])

    for item in data:
        folder_path = item[2]
        folder_name = re.search(r'dataset/(.*?)(\\|$)', folder_path)
        if folder_name:
            folder_key = folder_name.group(1)
            folder_sums[folder_key][0] += item[0]
            folder_sums[folder_key][1] += item[1]
            folder_sums[folder_key][2] += 1

    averages = {folder: [total[0] / total[2], total[1] / total[2]] for folder, total in folder_sums.items()}

    for item in data:
        folder_path = item[2]
        folder_name = re.search(r'dataset/(.*?)(\\|$)', folder_path)
        if folder_name:
            folder_key = folder_name.group(1)
            item[0] = averages[folder_key][0]
            item[1] = averages[folder_key][1]
    return data

        

In [18]:
def main():
    colume = []
    path = r"D:\Work\codev2\code\work_code\preprocess_data\fix"
    for folder in os.listdir(path):
        for json in os.listdir(os.path.join(path, folder)):
            if json.endswith(".json"):
                an_path = os.path.join(path, folder, json)
                masterbbox, chilrenbbox = get_bboxes(an_path)
                print(an_path)
                rows, cols = calculate_number_of_boxes(masterbbox, chilrenbbox)
                
                data = save_number_of_boxes(rows, cols, an_path)
                colume.append(data)



    data = avg_colume(colume)
    for i in range(len(data)):
        test = Test(data[i][2])
        grid = test.get_bboxes(rows=int(data[i][0]), cols=int(data[i][1]))
        test.save_squares_to_csv(grid, data[i][2][:-5] + ".csv")

main()

D:\Work\codev2\code\work_code\preprocess_data\fix\nzxc\343.json
[[616.6896551724138, 257.37931034482756], [640.166666666667, 3429.5358649789027], [4683.732067510549, 3397.784810126582], [4666.188679245283, 236.98113207547172]]
[[616.6896551724138, 257.37931034482756], [617.3241689965828, 343.1132712808836], [617.9586828207518, 428.84723221693974], [618.5931966449208, 514.5811931529959], [619.2277104690899, 600.3151540890519], [619.8622242932588, 686.049115025108], [620.4967381174279, 771.7830759611641], [621.1312519415968, 857.5170368972202], [621.7657657657659, 943.2509978332763], [622.4002795899348, 1028.9849587693325], [623.0347934141039, 1114.7189197053885], [623.6693072382728, 1200.4528806414446], [624.3038210624419, 1286.1868415775007], [624.9383348866108, 1371.9208025135567], [625.5728487107799, 1457.6547634496128], [626.2073625349489, 1543.3887243856689], [626.8418763591179, 1629.122685321725], [627.4763901832869, 1714.856646257781], [628.1109040074559, 1800.590607193837], [628